In [1]:
from dotenv import load_dotenv

load_dotenv("key.env",verbose=True)

True

# Language Models

### Completions API (Deprecated)

#### LLMs

In [2]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)


result = llm("自己紹介してください")
print(result)

/Users/tylerkurahashi/Documents/GitHub/langchain_study/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


### Chat Completions API

#### Chat models

- AIMessage
- HumanMessage
- SystemMessage

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？")
]
result = chat(messages)
print(result.content)


はい、おっしゃった通り、あなたの名前はジョンさんですね。他に何かお手伝いできることがありますか？


#### Streaming with Callbacks

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages) # type:ignore


はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問や会話に対応することができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。

# Prompts

## PromptTemplate

In [6]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください

料理名: {dish}
"""

prompt = PromptTemplate(
    input_variables=["dish"],
    template=template
)

result = prompt.format(dish="カレー")
print(result)


以下の料理のレシピを考えてください

料理名: カレー



## ChatPromptTemplate

In [9]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
        HumanMessagePromptTemplate.from_template("以下の料理のレシピを考えてください。\n\n料理名: {dish}"),
    ]
)

messages = chat_prompt.format_prompt(country="イギリス", dish="肉じゃが").to_messages()
print(messages)

[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。', additional_kwargs={}), HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが', additional_kwargs={}, example=False)]


# Output parsers
LLMの出力からPythonオブジェクトを作成する

## PydanticOutputParser

In [10]:
from pydantic import BaseModel, Field
from typing import List

class Recipe(BaseModel):
    ingredients: List[str] = Field(
        description="ingredients of the dish",
    )
    steps: List[str] = Field(
        description="steps to make the dish",
    )

In [11]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [12]:
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
